In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetLarge, ResNet101, DenseNet121
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.metrics import Precision, Recall

from keras.models import load_model

In [ ]:
IMAGE_PATH = '../input/resized-plant2021/img_sz_384'

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train

In [ ]:
train['labels']=train['labels'].apply( lambda string: string.split(' ') )
train.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
label= mlb.fit_transform(train['labels'])
print(label)

In [ ]:
labels = pd.DataFrame(label,columns=mlb.classes_,index=train.index)
labels

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255.0,
                            rotation_range=5,
                            zoom_range=0.1,
                            shear_range=0.05,
                            horizontal_flip=True,
                            validation_split=0.1)

train_generator = datagen.flow_from_dataframe(
    train,
    directory= IMAGE_PATH,
    subset='training',
    x_col='image',
    y_col='labels',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=444
    )
valid_generator = datagen.flow_from_dataframe(
    train,
    directory= IMAGE_PATH,
    subset='validation',
    x_col='image',
    y_col='labels',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=444
    )

In [ ]:
from keras.applications import DenseNet121
from keras.applications import DenseNet169

import keras
from keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_addons as tfa

model=DenseNet169(weights='imagenet',include_top=False, input_shape=(256,256,3))
x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(64,activation='relu')(x)
predictions=Dense(6,activation='sigmoid')(x)

DenseNetmodel=Model(inputs=model.input,outputs=predictions)

In [ ]:
DenseNetmodel.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
es=EarlyStopping(patience=5,monitor='val_loss',restore_best_weights=True)
lr = ReduceLROnPlateau(monitor= 'val_loss',mode='min',factor=0.01,patience=2,verbose=0)
hist = DenseNetmodel.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//64,
                    validation_steps=valid_generator.samples//64,
                    callbacks=[es])

In [ ]:
DenseNetmodel.save('densenet.h5')